In [49]:
from bokeh.models import ColumnDataSource, Label, LabelSet, Range1d
from bokeh.plotting import figure, output_file, show
import os
import geopandas as gpd
from pprint import pprint
import numpy as np
import pandas as pd
import sodapy as sp
import pandas_bokeh as pbk
from bokeh.io import output_notebook
from sodapy import Socrata
import matplotlib.pyplot as plt
import geopandas as gpd
import descartes 
from bokeh.plotting import figure, show
import pandas as pd
import sodapy as sd
import pandas_bokeh as pbk
pbk.output_notebook()
from bokeh.models import Legend
import matplotlib as mp

source = ColumnDataSource(data=dict(height=[66, 71, 72, 68, 58, 62],
                                    weight=[165, 189, 220, 141, 260, 174],
                                    names=['Mark', 'Amir', 'Matt', 'Greg',
                                           'Owen', 'Juan']))

p = figure(title='Dist. of 10th Grade Students at Lee High',
           x_range=Range1d(140, 275))
p.scatter(x='weight', y='height', size=8, source=source)
p.xaxis[0].axis_label = 'Weight (lbs)'
p.yaxis[0].axis_label = 'Height (in)'

labels = LabelSet(x='weight', y='height', text='names',
              x_offset=5, y_offset=5, source=source, render_mode='canvas')

citation = Label(x=70, y=70, x_units='screen', y_units='screen',
                 text='Collected by Luke C. 2016-04-01', render_mode='css',
                 border_line_color='black', border_line_alpha=1.0,
                 background_fill_color='white', background_fill_alpha=1.0)

p.add_layout(labels)
p.add_layout(citation)

#show(p)
source

Loading BokehJS ...

ColumnDataSource(id='5321', ...)

In [56]:
#gpd.points_from_xy(df_prov.Longitude, df_prov.Latitude)
mcp = gpd.read_file('data/divisions-administratives-v1r0-comarques-1000000-20210122.shp', crs='epsg:25831')
client = Socrata("analisi.transparenciacatalunya.cat", None)
results = client.get("rmgc-ncpb", limit=16774)
# Convert to pandas DataFrame
results_df =  gpd.GeoDataFrame.from_records(results)
# num d'accidents en funció de la comarca
com = pd.value_counts(results_df['nomcom'])
com = com.to_frame() # convertir a Data Frame
com = com.reset_index() # introduir un index (0, 1, 2...)
com = com.sort_values("index") # la columna de velocitat es diu index, ordenar-la
com = com.rename(columns={"index": "Comarca", "nomcom": "#accidents"}) # canviar els noms correctes
Com = gpd.GeoDataFrame(com).set_index("Comarca")
Com = Com.reset_index()
geom = mcp[["NOMCOMAR", "geometry"]] # escollir les columnes: comarca i geometria de mcp
geom = geom.sort_values('NOMCOMAR') # ordenar alfabèticament
Geom = gpd.GeoDataFrame(geom).set_index("NOMCOMAR") # tornem a posar l'índex
Geom = Geom.reset_index()
# seleccionar columna de geometries
geometry = Geom[['geometry']]
df_poblacion = pd.read_excel("data/PoblacioComarques.xls")
# Volem ordenar alfabèticament el dataset de poblacions perquè coincideixi amb el dataset de geometries de cada comarca
df_poblacion=df_poblacion.sort_values('comarca')
df_poblacion=pd.DataFrame(df_poblacion).set_index('comarca')
df_poblacion=df_poblacion.reset_index()
# seleccionar columnes pobl i pobl_percent
norm = df_poblacion[['pobl', 'pobl_percent']]
combo = pd.concat([Com,geometry,norm], axis=1)
combo['AccNorm'] = (combo['#accidents']/combo['pobl'])*1000
combo =  gpd.GeoDataFrame.from_records(combo)
combo.crs = "epsg:25831"

coord = pd.read_excel("Coordenades.xls")
coord_gdf = gpd.GeoDataFrame(coord, geometry=gpd.points_from_xy(coord.Longitud, coord.Latitud))
coord_gdf.crs = 'epsg:4326' # the database page indicates projection "WSG84", which corresponds to crs 'epsg:4326'
coord_gdf = coord_gdf.to_crs("epsg:4258")
coord_gdf.plot_bokeh(marker="inverted_triangle",size=10,color="yellow",show_figure=False)

source = ColumnDataSource(data=dict(height=coord_gdf['Latitud'],
                                    weight=coord_gdf['Longitud'],
                                    names=coord_gdf['Ciutat']))

p = combo.to_crs("epsg:4258").plot_bokeh(legend="Comarcas with car accident density", category='AccNorm',show_figure=False)

coord_gdf.to_crs("epsg:4258").plot_bokeh(figure=p,hovertool_string="""@City""",marker="inverted_triangle",size=8,color="yellow",show_figure=False)

labels = LabelSet(x='weight', y='height', text='names', x_offset=0, y_offset=0, source=source, render_mode='canvas')
p.add_layout(labels)

show(p)

C:\Users\Daniel\anaconda3\lib\site-packages\pandas_bokeh\geoplot.py:144: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for polygon in geometry:
